In [19]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import pymongo
# # importing webdriver from selenium
# from selenium import webdriver
# from selenium.webdriver.common.by import By

In [20]:
browser = Browser('chrome',headless=True)
scrapable = ['Processors', 'EthernetProducts']
scrape = scrapable[1]

In [21]:
url = 'https://ark.intel.com/content/www/us/en/ark.html#@'+scrape
csvfilename = f'Intel_{scrape}.csv'
baseurl = 'https://ark.intel.com/'

browser.visit(url)

rootHTML = bs(browser.html, 'html.parser')

In [22]:
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

In [23]:
client = pymongo.MongoClient('localhost', 27017)
db = client.IntelScrape
collection = db.items

In [24]:
ethernet_name = []

In [25]:
for div in rootHTML.find_all('div', scrape):
    for link in rootHTML.\
        find_all('div', {'data-parent-panel-key':div['data-panel-key']})[0].\
        find_all('a'):
        browser.visit(baseurl+link['href'])
        time.sleep(1)
        for td in bs(browser.html, 'html.parser').find_all('td', class_='ark-product-name'):
            browser.visit(baseurl+td.a['href'])
            time.sleep(1)
            data = {'scrape':scrape}
            try:
                for c in bs(browser.html, 'html.parser').\
                    find_all('div', class_='arkProductSpecifications')[0].\
                    find_all('span', class_=['label','value']):
                    if c['class'][0] == 'label':
                        temp = c.text.strip().replace('.','_')
                            
                    else:
                            
                        data[temp] = c.text.strip()
                            #print(data)
                            #processor_list.append(data[temp])
                            #data[temp].append(c.text.strip())
                            #wrapper.update(data)
                                 
            except:
                print('failed '+link['href'])
                continue
                
                #df = df.concat([df, processor_list], axis=1)
                #wrapper.update(data)
               
                #insert_data_into_database(data)
            collection.insert_one(data)
            ethernet_name.append(td.a.text)
            print(td.a.text, end='; ')

Intel® Ethernet Network Adapter E810-CQDA2T; Intel® Ethernet Network Adapter E810-2CQDA2; Intel® Ethernet Network Adapter E810-CQDA1; Intel® Ethernet Network Adapter E810-CQDA1 for OCP; Intel® Ethernet Network Adapter E810-CQDA1 for OCP 3.0; Intel® Ethernet Network Adapter E810-CQDA2; Intel® Ethernet Network Adapter E810-CQDA2 for OCP 3.0; Intel® Ethernet Network Adapter E810-XXVDA4T; Intel® Ethernet Network Adapter E810-XXVDA4 for OCP 3.0; Intel® Ethernet Network Adapter E810-XXVDA2; Intel® Ethernet Network Adapter E810-XXVDA2 for OCP 3.0; Intel® Ethernet Network Adapter E810-XXVDA4; Intel® Ethernet Controller E810-XXVAM2; Intel® Ethernet Controller E810-CAM1; Intel® Ethernet Controller E810-CAM2; Intel® Ethernet Network Adapter I710-T4L; Intel® Ethernet Network Adapter I710-T4L for OCP 3.0; Intel® Ethernet Network Adapter XXV710-DA2T; Intel® Ethernet Network Adapter XXV710-DA2 for OCP; Intel® Ethernet Network Adapter XXV710-DA1; Intel® Ethernet Network Adapter XXV710-DA1 for OCP; Int

In [28]:
len(ethernet_name)

185

In [27]:
import csv
with open(csvfilename,'w',newline='') as csvfile:
    cols = []
    data = db.items.find({'scrape':{'$eq':scrape}})
    for i in data:
        cols.extend([x for x in i.keys() if x not in cols])
    cols.remove('_id')
    cols.remove('scrape')
    
    writer = csv.DictWriter(csvfile, cols)
    writer.writeheader()
    for i in data.rewind():
        i.pop('_id')
        i.pop('scrape')
        writer.writerow(i)

In [29]:
import pandas as pd

In [32]:
df_intel_ethernet = pd.read_csv("IntelArk3EthernetProducts.csv", dtype='str')

In [33]:
df_intel_ethernet

,Product Collection,Marketing Status,Launch Date,Vertical Segment,Cable Medium,Cabling Type,Bracket Height,Supported Operating Systems,New Design Availability Expiration Date,Recommended Customer Price,...,IPv4/IPv6 Routes,Intel® FlexPipe™ Technology,Advanced Load Balancing,CEE/DCB Features,Server Virtualization Support,Advanced Tunneling Features,Carrier Ethernet Support,CPU Interface,Applications,Extended Temperature Options
0,100GbE Intel® Ethernet Network Adapter E810,Launched,Q3'22,Server,"Copper, Fiber, SiPh","QSFP28 port - DAC, Optics, AOC's. SiPh",Full Height,View now,"Sunday, January 1, 2040",$901.00-$1026.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100GbE Intel® Ethernet Network Adapter E810,Launched,Q1'21,Server,NaN,"QSFP28 - DAC, Optics, and AOC's",Full Height,View now,"Sunday, January 1, 2040",$692.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100GbE Intel® Ethernet Network Adapter E810,Launched,Q4'20,Server,NaN,"QSFP28 port - DAC, Optics, and AOC's",Low Profile and Full Height,View now,"Sunday, January 1, 2040",$400.00-$416.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100GbE Intel® Ethernet Network Adapter E810,Discontinued,Q4'20,Server,NaN,QSFP28 port - DAC,OCP 2.0,View now,"Sunday, January 1, 2040",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100GbE Intel® Ethernet Network Adapter E810,Launched,Q1'21,Server,NaN,"QSFP28 port - DAC, Optics, AOC's",OCP 3.0,View now,"Sunday, January 1, 2040",$416.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Intel® Fast Ethernet Network Connections,Discontinued,Q2'00,NaN,NaN,NaN,NaN,NaN,"Friday, April 1, 2005",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,Intel® Fast Ethernet Network Connections,Discontinued,Q2'00,NaN,NaN,NaN,NaN,NaN,"Friday, April 1, 2005",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,Intel® Fast Ethernet Network Connections,Discontinued,Q4'99,NaN,NaN,NaN,NaN,NaN,"Monday, October 4, 2004",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,Intel® PRO/100 S Server Adapter Series,Discontinued,Q2'02,Server,Copper,Category-5 up to 100m,Full Height,NaN,"Wednesday, October 26, 2022",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Scrape processor
